# Annotate Putative Somatic Variants with CIViC and VEP

Welcome back to OpenCAP Jupyter Notebooks! Here you can run pre-designed code to generate an output report for your variants. 

To run Jupyter notebook cells hold-down on "shift" and select "enter".

## Commands to annotate variants

We have created a python script called identified_variants_to_annotation.py that will take in variants from the sequencing pipeline and output a document with annotation information. The output will include variant effect predictions and clinical interpretations of variants in cancer.

Before running this cell, please upload your somatic variants to the home directory (see ReadTheDocs) and change the input variant list name to match your somatic variant list. You can run the cell by holding the "shift" key and selecting "enter".

In [ ]:
#run command  python code             #input variant list
%run -i 'identified_variants_to_annotation.py' 'test_annotate_variants.tsv'

In [19]:
from docx import Document
from docx.shared import Inches
import pandas as pd

In [30]:
sample_name = 'SCLC_5'
CIViC_annotation_length = 3

In [42]:
CIViC_annotation = pd.DataFrame([['KRAS', 'MUTATION','Supports Resistance to Cetuximab for patients with Colorectal Cancer','CIViC EID6464','PMID:22722830'],
                                ['KRAS', 'EXON 2 MUTATION', 'Supports Resistance to EGFR Inhibitor for patients with Colorectal Cancer','CIViC EID1208','PMID:27037411']])
CIViC_annotation.columns=['Gene', 'Variant', 'Description', 'CIViC Link', 'PubMedID Link']

In [46]:
CIViC_annotation

,Gene,Variant,Description,CIViC Link,PubMedID Link
0,KRAS,MUTATION,Supports Resistance to Cetuximab for patients ...,CIViC EID6464,PMID:22722830
1,KRAS,EXON 2 MUTATION,Supports Resistance to EGFR Inhibitor for pati...,CIViC EID1208,PMID:27037411


In [45]:
document = Document()
document.add_picture('Extra/report_header.png', width=Inches(6))
document.add_heading('SOMATIC VARIANT ANNOTATION', 0)

p = document.add_paragraph('')

document.add_heading('Variant Effect Predictions', level=1)
   
p = document.add_paragraph()
p.add_run('Sample Name:').bold = True
p.add_run(str(sample_name))



table = document.add_table(rows=1, cols=3)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Qty'
hdr_cells[1].text = 'Id'
hdr_cells[2].text = 'Desc'
# for qty, id, desc in records:
#     row_cells = table.add_row().cells
#     row_cells[0].text = str(qty)
#     row_cells[1].text = id
#     row_cells[2].text = desc
    
    
document.add_heading('CIViC Variant Annotations', level=1)
 
    
table = document.add_table(rows=len(), cols=5)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Gene'
hdr_cells[1].text = 'Variant'
hdr_cells[2].text = 'Description'
hdr_cells[3].text = 'CIViC Link'
hdr_cells[4].text = 'PubMedID Link'
for i, row in CIViC_annotation.iterrows():
    row_cells = table.add_row().cells
    row_cells[0].text = row['Gene']
    row_cells[1].text = row['Variant']
    row_cells[2].text = row['Description']
    row_cells[1].text = row['CIViC Link']
    row_cells[2].text = row['PubMedID Link']



p = document.add_paragraph('OpenCAP is intended for research use only and clinical applications of subsequent panels designed using the SOP would require further panel validation.')

document.save('demo.docx')